To start sampling America by the county, we need to generate some data points (latitude and longitude pairs) to use. In this notebook, we generate a list of latitutes and longitudes using the Latin Square technique. We am given a latitude and longitude in the center of each county and the total square area of each county from the Gazetter Dataset which you can download <a href=
'http://people.bu.edu/balawson/csv/congress.csv'>here</a>.I use the Latin Square technique to randomally sample the county. I use PyDOE to implement this and you can read more about Python Design Of Experiments here: http://pythonhosted.org/pyDOE/randomized.html. I'll be using Pandas to manage data. 

In [1]:
import pandas as pd
import random
from pyDOE import *
import math

Here's the link to Wikipedia's page on Latin Hypercube sampling. In an oversimplified way, I link to think of it as "Sudoku" for sampling - every row and column has exactly one sample (at least in orthogonal latin sampling) Orthogonal latin square sampling is not implemented in PyDOE, so I don't use it. (https://en.wikipedia.org/wiki/Latin_hypercube_sampling) I want numbers that will modify a given latitude and longitude such that the new point is somewhere within a boundary surrounding the given point. 

In [2]:
#orginal data source: https://www.census.gov/geo/maps-data/data/gazetteer2014.html
df = pd.DataFrame.from_csv("csv/congress.csv")
df.index = df.GEOID #because current index is just state abbreviation 
#grab the first row and look at it - what info do we want to grab? ALAND, INTPTLAT, and INTPTLONG
df.iloc[0:1]

,GEOID,ANSICODE,NAME,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
GEOID,,,,,,,,,
1001,1001,161526,Autauga County,1539584444,25773561,594.437,9.951,32.53216,-86.646469


In [3]:
a = lhs(2, samples=20, criterion='center')
a

array([[ 0.525,  0.375],
       [ 0.325,  0.525],
       [ 0.875,  0.225],
       [ 0.675,  0.825],
       [ 0.075,  0.275],
       [ 0.825,  0.425],
       [ 0.225,  0.875],
       [ 0.425,  0.675],
       [ 0.625,  0.725],
       [ 0.125,  0.025],
       [ 0.975,  0.075],
       [ 0.025,  0.775],
       [ 0.725,  0.475],
       [ 0.775,  0.325],
       [ 0.275,  0.575],
       [ 0.175,  0.925],
       [ 0.475,  0.175],
       [ 0.575,  0.975],
       [ 0.925,  0.125],
       [ 0.375,  0.625]])

In [4]:
#renormalize values between -1 and 1 in order to plot on all four quadrants
b = (a-0.5)*2
b

array([[ 0.05, -0.25],
       [-0.35,  0.05],
       [ 0.75, -0.55],
       [ 0.35,  0.65],
       [-0.85, -0.45],
       [ 0.65, -0.15],
       [-0.55,  0.75],
       [-0.15,  0.35],
       [ 0.25,  0.45],
       [-0.75, -0.95],
       [ 0.95, -0.85],
       [-0.95,  0.55],
       [ 0.45, -0.05],
       [ 0.55, -0.35],
       [-0.45,  0.15],
       [-0.65,  0.85],
       [-0.05, -0.65],
       [ 0.15,  0.95],
       [ 0.85, -0.75],
       [-0.25,  0.25]])

In [5]:
#http://gis.stackexchange.com/questions/2951/algorithm-for-offsetting-a-latitude-longitude-by-some-amount-of-meters
def meters_to_degs(x, y):
    #takes meters in the x- and y-directions
    #returns a tuple changes in degree
    #this method is refered to as 'quick and dirty' and not suggested for life-dependent applications or long distances
    return ((y/111111.0), x/(111111 * math.cos(y)))

In [6]:
def get_max_distances(land_area):
    #assuming counties are square (smaller area than circle - less points near or outside boundary)
    side = math.sqrt(land_area)
    r = side/2
    return r

In [7]:
def get_max_distances_circle(land_area):
    #assuming counties are circles (which they are not, but shapes are hard)
    r_2 = land_area/math.pi
    r = math.sqrt(r_2)
    return r

In [8]:
def get_degree_ranges(land_area):
    d = get_max_distances(land_area)
    return (meters_to_degs(d, d))

In [12]:
#let's test the functions I wrote using the first entry in the csv
x = df.iloc[0:1].ALAND
al = get_degree_ranges(x)
al

(0.17656910076302565, -0.1987309136349029)

The general idea here is to find the maxium distance from the center in the x and y direction and then find random samples within that boundary. Since counties are irregular in shape we will have to do some cleaning.

In [13]:
#let's try to look at the info that's interesting at the moment (1001 corresponds to the first GEOID/index value)
df["INTPTLONG"][1001]

KeyError: 'INTPTLONG'

In [14]:
#why the error message? try this one:
df["INTPTLONG                                                                                                               "][1001]

-86.646468999999996

In [15]:
#quick fix
df.columns = [x.strip() for x in df.columns]
df["INTPTLONG"][1001]

-86.646468999999996

In [16]:
def sampler(row, val):
    #row corresponds to one of the dataframe rows
    #val is the row of the Latin Square that I will use for this sample
    latin_square_coefficient = b[val]
    multiplier = get_degree_ranges(row.ALAND)
    center = [row.INTPTLAT, row.INTPTLONG]
    return latin_square_coefficient*multiplier + center

In [17]:
latin_square_coefficient = lhs(2, samples=20, criterion='center')
latin_square_coefficient = (latin_square_coefficient-0.5)*2

Let's test out the sampling function using the first row of the dataframe.

In [18]:
for x in xrange(20):
    print str(sampler(df.loc[1001], x)).replace('[','').replace(']','')

 32.54098846 -86.59678627
 32.47036081 -86.65640555
 32.66458683 -86.537167  
 32.59395919 -86.77564409
 32.38207626 -86.55704009
 32.64692992 -86.61665936
 32.43504699 -86.79551719
 32.50567463 -86.71602482
 32.57630228 -86.73589791
 32.39973317 -86.45767463
 32.69990065 -86.47754772
 32.36441935 -86.755771  
 32.6116161  -86.63653245
 32.62927301 -86.57691318
 32.4527039  -86.67627864
 32.41739008 -86.81539028
 32.52333154 -86.51729391
 32.55864537 -86.83526337
 32.68224374 -86.49742081
 32.48801772 -86.69615173


To get a general idea that this is working I use Mod-Rule (http://www.mob-rule.com/gmap) to plot a point and see if it falls in the right county. I then plotted a bunch using  (http://www.mapcustomizer.com/) to get a general idea. You can try coping and pasting the results above. Quick and dirty: to see if we are on the right track. Looks like a few points are out of the range, but for the most part usable. I'm going to generate samples for every county and save this in another dataframe. Then I'm going to remove the bad samples. <br>
<img src='./img/sampling1.png' height="350" width="350" align='left'><img src="./img/sampling2.png" height="350" width="350" align='right'>

In [17]:
samples = pd.DataFrame(columns=[x for x in range(0,20)])

In [18]:
for idx, row in df.iterrows():
    for x in range(len(b)):
        samples.loc[idx,x] = (sampler(row, x))

In [19]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 1001 to 56045
Data columns (total 20 columns):
0     3142 non-null object
1     3142 non-null object
2     3142 non-null object
3     3142 non-null object
4     3142 non-null object
5     3142 non-null object
6     3142 non-null object
7     3142 non-null object
8     3142 non-null object
9     3142 non-null object
10    3142 non-null object
11    3142 non-null object
12    3142 non-null object
13    3142 non-null object
14    3142 non-null object
15    3142 non-null object
16    3142 non-null object
17    3142 non-null object
18    3142 non-null object
19    3142 non-null object
dtypes: object(20)

In [37]:
for x in samples.loc[1001]:
    print str(x).replace('-',',-').replace(' ','')

[32.4527039,-86.67627864]
[32.57630228,-86.79551719]
[32.50567463,-86.69615173]
[32.69990065,-86.57691318]
[32.38207626,-86.61665936]
[32.36441935,-86.73589791]
[32.47036081,-86.77564409]
[32.59395919,-86.59678627]
[32.41739008,-86.81539028]
[32.55864537,-86.51729391]
[32.43504699,-86.63653245]
[32.68224374,-86.71602482]
[32.6116161,-86.537167]
[32.64692992,-86.49742081]
[32.66458683,-86.65640555]
[32.52333154,-86.45767463]
[32.54098846,-86.55704009]
[32.39973317,-86.47754772]
[32.62927301,-86.755771]
[32.48801772,-86.83526337]


In [21]:
samples.to_csv('csv/samples.csv')

To check the accurarcy of the lat/lng and FIPS (the county code) assignment I'm using <a href="http://www.datasciencetoolkit.org/"> Data Science Toolkit</a> and their dope "Coordinates to Politics" API. They are also super awesome and have an <a href="http://www.datasciencetoolkit.org/developerdocs#amazon">AWS AMI (Amazon Web Service Amazon Machine Image)</a> so I can spin up my own EC2 instance with all their services - so I don't spam them to infinity. You can also download their entire server (via torrent or direct download). 

In [31]:
#http://www.census.gov/popest/about/terms.html

In [32]:
# Code for setting the style of the notebook

from IPython.core.display import HTML
def css_styling():
    styles = open("./theme/custom.css", "r").read()
    return HTML(styles)
css_styling()